<a href="https://colab.research.google.com/github/Xawad11/Movie-recommendation-system/blob/main/ML_PROJECT_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy scikit-learn pyarrow


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

PROJECT_DIR = "/content/drive/MyDrive/ML Project"   # note the space in the folder name
os.makedirs(PROJECT_DIR, exist_ok=True) # Make sure this folder exists (creates if not already there)

CSV_PATH = f"{PROJECT_DIR}/movies.csv"

if os.path.exists(CSV_PATH):
    print(" Dataset found:", CSV_PATH)
else:
    print(" File not found at", CSV_PATH)


 Dataset found: /content/drive/MyDrive/ML Project/movies.csv


In [ ]:
# Load dataset into pandas and check structure

import pandas as pd

# Load CSV into a DataFrame
df_raw = pd.read_csv(CSV_PATH)


print("Dataset shape:", df_raw.shape)

# first 5 rows
df_raw.head()


Dataset shape: (4803, 24)


,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
#  Normalize column names for easier access

df = df_raw.copy()
df.columns = [c.lower() for c in df.columns]  # c.lower() make all column names lowercase

print("Columns after normalization:", list(df.columns))


Columns after normalization: ['index', 'budget', 'genres', 'homepage', 'id', 'keywords', 'original_language', 'original_title', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count', 'cast', 'crew', 'director']


In [ ]:
# If a column is missing, this will raise a error

import pandas as pd
import numpy as np

# Define the columns we expect to use in later steps
TEXT_COLS = ["title", "overview", "genres", "keywords", "cast", "director"]
NUM_COLS  = ["vote_average", "vote_count"]

# Check if they exist
missing_text = [c for c in TEXT_COLS if c not in df.columns]
missing_num  = [c for c in NUM_COLS if c not in df.columns]

if missing_text:
    raise ValueError(f"Missing TEXT columns: {missing_text}. Please add orrename in CSV or adjust TEXT_COLS")
# Numeric columns are optional and define as NaN if missing
for c in missing_num:
    print(f"Note: numeric column '{c}' not found. Creating as NaN")
    df[c] = np.nan

# deal with missing texts (prevents issues in TF-IDF later)
for c in TEXT_COLS:
    df[c] = df[c].fillna("").astype(str).str.strip()

# convert to nums (strings → numbers; bad values → NaN)
for c in NUM_COLS:
    df[c] = pd.to_numeric(df[c], errors="coerce")


print("Fixed-schema setup complete")
print("Text columns:", TEXT_COLS)
print("Numeric columns:", NUM_COLS)
print("Sample rows:")
display(df[TEXT_COLS + NUM_COLS].head(5))


Fixed-schema setup complete
Text columns: ['title', 'overview', 'genres', 'keywords', 'cast', 'director']
Numeric columns: ['vote_average', 'vote_count']
Sample rows:


,title,overview,genres,keywords,cast,director,vote_average,vote_count
0,Avatar,"In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron,7.2,11800
1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski,6.9,4500
2,Spectre,A cryptic message from Bond’s past sends him o...,Action Adventure Crime,spy based on novel secret agent sequel mi6,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes,6.3,4466
3,The Dark Knight Rises,Following the death of District Attorney Harve...,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan,7.6,9106
4,John Carter,"John Carter is a war-weary, former military ca...",Action Adventure Science Fiction,based on novel mars medallion space travel pri...,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton,6.1,2124


In [ ]:
# Parse the `genres` column into a clean list of tags per movie

import ast
import re

def parse_genres(cell: str):
    """
    Convert a 'genres' cell into a list of genre tags
    Handles common formats:
       JSON-like lists:      ["Action","Adventure"]  OR  [{"id":..,"name":"Action"}, ...]
       Delimited strings:    'Action|Adventure' or 'Action, Adventure' or 'Action/Adventure'
       Simple short strings: 'Action' (single token)
    Returns [] if nothing valid is found
    """
    s = str(cell).strip()
    if not s or s.lower() in {"nan", "none"}:
        return []

    # JSON-like list (strings or dicts w/ 'name')
    if s.startswith("[") and s.endswith("]"):
        try:
            obj = ast.literal_eval(s)
            if isinstance(obj, list):
                out = []
                for el in obj:
                    if isinstance(el, dict) and "name" in el:
                        out.append(str(el["name"]).strip())
                    elif isinstance(el, str):
                        out.append(el.strip())
                return [g for g in out if g]
        except Exception:
            pass  # fall through to other parsing strategies

    # Delimiter-based
    for sep in ["|", ",", ";", "/"]:
        if sep in s:
            parts = [p.strip() for p in s.split(sep)]
            return [p for p in parts if p]

    # Short single string (avoid splitting long sentences)
    if len(s.split()) <= 3:
        return [p.strip() for p in s.split() if p.strip()]

    # Otherwise, treat as unparseable
    return []

# Apply the parser to create a canonical list column
df["genre_list"] = df["genres"].apply(parse_genres)


n_with_genre = int((df["genre_list"].str.len() > 0).sum())
print(f"Rows with at least one parsed genre: {n_with_genre} / {len(df)}")


examples = df.loc[df["genre_list"].str.len() > 0, ["title", "genres", "genre_list"]].head(5)
display(examples)



Rows with at least one parsed genre: 3681 / 4803


,title,genres,genre_list
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,"[Adventure, Fantasy, Action]"
2,Spectre,Action Adventure Crime,"[Action, Adventure, Crime]"
5,Spider-Man 3,Fantasy Action Adventure,"[Fantasy, Action, Adventure]"
6,Tangled,Animation Family,"[Animation, Family]"
8,Harry Potter and the Half-Blood Prince,Adventure Fantasy Family,"[Adventure, Fantasy, Family]"


In [ ]:
################################### FIX ###############

# Restore genres & set TF-IDF alias (non-destructive)

import re
import pandas as pd

#  Restore the original genres text from the data loaded before
#  If CSV already had plain space-separated genres like: "Action Adventure Science Fiction Thriller" — KEEPING THEM AS-IS

src_col = None
for c in ["genres", "genre", "genre_names", "genre_list"]:   # Different words verify if we use in a different dataset with diff genre column name
    if c in df_raw.columns:
        src_col = c
        break

if src_col is not None:
    df["genres"] = df_raw[src_col].astype(str).fillna("").str.strip()
else:
    # If df_raw didn't keep the original column, just keep current df['genres']
    print("Note: couldn't find original genres in df_raw, keeping current df['genres'].")


df["genres_display"] = df["genres"]

######################
#  Making a TF-IDF-safe column by protecting multi-word genres ONLY (VERY IMPORTANT)
#  This prevents splitting "Science Fiction" into separate tokens.
def protect_multiword(s: str):
    s = str(s)
    s = re.sub(r"\bScience Fiction\b", "Science_Fiction", s, flags=re.IGNORECASE)
    s = re.sub(r"\bTV Movie\b", "TV_Movie", s, flags=re.IGNORECASE)
    s = re.sub(r"\bFilm Noir\b", "Film_Noir", s, flags=re.IGNORECASE)
    return s

df["genres_tfidf"] = df["genres"].apply(protect_multiword)




display(df[["title", "genres", "genres_display", "genres_tfidf"]].head(8))


,title,genres,genres_display,genres_tfidf
0,Avatar,Action Adventure Fantasy Science Fiction,Action Adventure Fantasy Science Fiction,Action Adventure Fantasy Science_Fiction
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,Adventure Fantasy Action,Adventure Fantasy Action
2,Spectre,Action Adventure Crime,Action Adventure Crime,Action Adventure Crime
3,The Dark Knight Rises,Action Crime Drama Thriller,Action Crime Drama Thriller,Action Crime Drama Thriller
4,John Carter,Action Adventure Science Fiction,Action Adventure Science Fiction,Action Adventure Science_Fiction
5,Spider-Man 3,Fantasy Action Adventure,Fantasy Action Adventure,Fantasy Action Adventure
6,Tangled,Animation Family,Animation Family,Animation Family
7,Avengers: Age of Ultron,Action Adventure Science Fiction,Action Adventure Science Fiction,Action Adventure Science_Fiction


In [ ]:
# Building TF-IDF FEATURES from multiple text fields

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer

# Define which text columns we use and how much weight each should have
# tweaking WEIGHTS later during ablation (overview is most important)
weights = {
    "overview": 0.6,
    "genres_tfidf": 0.2,
    "keywords": 0.15,
    "cast":     0.03,
    "director": 0.02
}

# Create a ColumnTransformer that applies TF-IDF to each text column
transformers = []
for col in weights.keys():
    transformers.append((
        f"tfidf_{col}",  # Dataset cleaning time
        TfidfVectorizer(stop_words="english", ngram_range=(1,2), min_df=2, max_features=50000), # drop very common words (the,is,a,and etc).|Unigram,Bigram dealing "space","opera" and "space opera"|Drop words appearing in only one movie|50,000 most important tokens
        col
    ))

vectorizer = ColumnTransformer(transformers, transformer_weights={f"tfidf_{c}": w for c, w in weights.items()})

# Fit and transform on the whole dataset
X_tfidf = vectorizer.fit_transform(df)

print("TF-IDF shape:", X_tfidf.shape)  # MOVIE x FEATURE MATRIX, After cleaning only 32660 valid tokens < 50000


TF-IDF shape: (4803, 32669)


In [ ]:
######  Build COSINE SIMILARITY recommender with Nearest Neighbors

from sklearn.neighbors import NearestNeighbors

# Use cosine distance
nn_model = NearestNeighbors(metric="cosine", algorithm="brute") #brute force cause dataset not so big
nn_model.fit(X_tfidf)

# Helper to get recommendations by title
title_to_index = {t.lower(): i for i, t in enumerate(df["title"].astype(str))}   # lower() so avatar and Avatar both works, also title lookup dictionary

def recommend_by_title(title, k=10):
    """Recommend top-k similar movies for a given title using cosine similarity."""
    idx = title_to_index.get(title.lower())
    if idx is None:
        raise ValueError(f"Title '{title}' not found in dataset")

    distances, indices = nn_model.kneighbors(X_tfidf[idx], n_neighbors=k+1)  # +1 to drop the movie itself
    indices, distances = indices[0][1:], distances[0][1:]  # remove the first (self)

    recs = df.iloc[indices][["title","genres_display","vote_average","vote_count"]].copy()
    recs.insert(0, "similarity", 1 - distances)  # cosine sim = 1 - distance
    return recs




In [ ]:
# Try a movie
print("Recommendations for 'Avatar':")
display(recommend_by_title("Avatar", k=10))

# Another test movie
print("Recommendations for 'The Dark Knight':")
display(recommend_by_title("The Dark Knight", k=10))

# If unsure of the exact title, search for it
search_term = "Harry Potter"
matches = [t for t in df["title"] if search_term.lower() in str(t).lower()]
print("Matches found for:", search_term, "->", matches[:10])  # shows up to 10 possible matches

# Then pass the exact title you want into recommend_by_title
if matches:
    display(recommend_by_title(matches[0], k=10))


Recommendations for 'Avatar':


,similarity,title,genres_display,vote_average,vote_count
1341,0.198955,The Inhabited Island,Action Fantasy Science Fiction Thriller,5.3,23
634,0.175673,The Matrix,Action Science Fiction,7.9,8907
3604,0.154280,Apollo 18,Horror Thriller Science Fiction,5.0,356
529,0.143492,Tears of the Sun,Action Drama War,6.4,573
775,0.134419,Supernova,Horror Science Fiction Thriller,4.9,109
2130,0.130784,The American,Crime Drama Thriller,5.8,481
107,0.107551,Warcraft,Action Adventure Fantasy,6.3,2268
311,0.107200,The Adventures of Pluto Nash,Action Comedy Science Fiction,4.4,142
36,0.102727,Transformers: Age of Extinction,Science Fiction Action Adventure,5.8,3095
1274,0.102275,Just Visiting,Comedy Fantasy Science Fiction,4.8,171


Recommendations for 'The Dark Knight':


,similarity,title,genres_display,vote_average,vote_count
3,0.363535,The Dark Knight Rises,Action Crime Drama Thriller,7.6,9106
299,0.273210,Batman Forever,Action Crime Fantasy,5.2,1498
428,0.228259,Batman Returns,Action Fantasy,6.6,1673
1359,0.195593,Batman,Fantasy Action,7.0,2096
119,0.195356,Batman Begins,Action Crime Drama,7.5,7359
3854,0.177763,"Batman: The Dark Knight Returns, Part 2",Action Animation,7.9,419
879,0.150584,Law Abiding Citizen,Drama Crime Thriller,7.2,1486
2507,0.146041,Slow Burn,Mystery Crime Drama Thriller,5.5,16
205,0.143364,Sherlock Holmes: A Game of Shadows,Adventure Action Crime Mystery,7.0,3886
1181,0.140564,JFK,Drama Thriller History,7.5,502


Matches found for: Harry Potter -> ['Harry Potter and the Half-Blood Prince', 'Harry Potter and the Order of the Phoenix', 'Harry Potter and the Goblet of Fire', 'Harry Potter and the Prisoner of Azkaban', "Harry Potter and the Philosopher's Stone", 'Harry Potter and the Chamber of Secrets']


,similarity,title,genres_display,vote_average,vote_count
114,0.426471,Harry Potter and the Goblet of Fire,Adventure Fantasy Family,7.5,5608
113,0.305199,Harry Potter and the Order of the Phoenix,Adventure Fantasy Family Mystery,7.4,5494
191,0.211007,Harry Potter and the Prisoner of Azkaban,Adventure Fantasy Family,7.7,5877
276,0.209354,Harry Potter and the Chamber of Secrets,Adventure Fantasy Family,7.4,5815
197,0.183938,Harry Potter and the Philosopher's Stone,Adventure Fantasy Family,7.5,7006
4133,0.124199,Plush,Thriller,5.4,40
4756,0.109013,The Call of Cthulhu,Horror Thriller Fantasy,6.9,41
341,0.107147,Percy Jackson & the Olympians: The Lightning T...,Adventure Fantasy Family,6.0,2010
1939,0.105142,BloodRayne,Action Adventure Fantasy Horror,3.5,118
501,0.102377,The Little Prince,Adventure Animation Fantasy,7.6,756


In [ ]:
#KNN Recommender

In [ ]:
# Build KNN index over TF-IDF vectors

from sklearn.neighbors import NearestNeighbors

# KNN with cosine distance, brute force is fine for ~5k movies
knn_model = NearestNeighbors(metric="cosine", algorithm="brute")
knn_model.fit(X_tfidf)

print(" KNN model fitted on TF-IDF vectors")


 KNN model fitted on TF-IDF vectors


In [ ]:
# Recommend using KNN index

def knn_recommend_by_title(title, k=10):
    """Recommend k movies using KNN search (cosine distance)."""
    idx = title_to_index.get(title.lower())
    if idx is None:
        raise ValueError(f"Title '{title}' not found in dataset")

    distances, indices = knn_model.kneighbors(X_tfidf[idx], n_neighbors=k+1)
    indices, distances = indices[0][1:], distances[0][1:]  # drop the movie itself

    recs = df.iloc[indices][["title","genres_tfidf","vote_average","vote_count"]].copy()
    recs.insert(0, "similarity", 1 - distances)
    return recs



In [ ]:
# Example 1: Try a popular movie
print("KNN Recommendations for 'Avatar':")
display(knn_recommend_by_title("Avatar", k=10))

# Example 2: Another test movie
print("KNN Recommendations for 'The Dark Knight':")
display(knn_recommend_by_title("The Dark Knight", k=10))

# Example 3: If you're unsure of the exact title, search for it
search_term = "Harry Potter"
matches = [t for t in df["title"] if search_term.lower() in str(t).lower()]
print("Matches found for:", search_term, "->", matches[:10])  # shows up to 10 possible matches

# Then pass the exact title you want into knn_recommend_by_title
if matches:
    display(knn_recommend_by_title(matches[0], k=10))


KNN Recommendations for 'Avatar':


,similarity,title,genres_tfidf,vote_average,vote_count
1341,0.198955,The Inhabited Island,Action Fantasy Science_Fiction Thriller,5.3,23
634,0.175673,The Matrix,Action Science_Fiction,7.9,8907
3604,0.154280,Apollo 18,Horror Thriller Science_Fiction,5.0,356
529,0.143492,Tears of the Sun,Action Drama War,6.4,573
775,0.134419,Supernova,Horror Science_Fiction Thriller,4.9,109
2130,0.130784,The American,Crime Drama Thriller,5.8,481
107,0.107551,Warcraft,Action Adventure Fantasy,6.3,2268
311,0.107200,The Adventures of Pluto Nash,Action Comedy Science_Fiction,4.4,142
36,0.102727,Transformers: Age of Extinction,Science_Fiction Action Adventure,5.8,3095
1274,0.102275,Just Visiting,Comedy Fantasy Science_Fiction,4.8,171


KNN Recommendations for 'The Dark Knight':


,similarity,title,genres_tfidf,vote_average,vote_count
3,0.363535,The Dark Knight Rises,Action Crime Drama Thriller,7.6,9106
299,0.273210,Batman Forever,Action Crime Fantasy,5.2,1498
428,0.228259,Batman Returns,Action Fantasy,6.6,1673
1359,0.195593,Batman,Fantasy Action,7.0,2096
119,0.195356,Batman Begins,Action Crime Drama,7.5,7359
3854,0.177763,"Batman: The Dark Knight Returns, Part 2",Action Animation,7.9,419
879,0.150584,Law Abiding Citizen,Drama Crime Thriller,7.2,1486
2507,0.146041,Slow Burn,Mystery Crime Drama Thriller,5.5,16
205,0.143364,Sherlock Holmes: A Game of Shadows,Adventure Action Crime Mystery,7.0,3886
1181,0.140564,JFK,Drama Thriller History,7.5,502


Matches found for: Harry Potter -> ['Harry Potter and the Half-Blood Prince', 'Harry Potter and the Order of the Phoenix', 'Harry Potter and the Goblet of Fire', 'Harry Potter and the Prisoner of Azkaban', "Harry Potter and the Philosopher's Stone", 'Harry Potter and the Chamber of Secrets']


,similarity,title,genres_tfidf,vote_average,vote_count
114,0.426471,Harry Potter and the Goblet of Fire,Adventure Fantasy Family,7.5,5608
113,0.305199,Harry Potter and the Order of the Phoenix,Adventure Fantasy Family Mystery,7.4,5494
191,0.211007,Harry Potter and the Prisoner of Azkaban,Adventure Fantasy Family,7.7,5877
276,0.209354,Harry Potter and the Chamber of Secrets,Adventure Fantasy Family,7.4,5815
197,0.183938,Harry Potter and the Philosopher's Stone,Adventure Fantasy Family,7.5,7006
4133,0.124199,Plush,Thriller,5.4,40
4756,0.109013,The Call of Cthulhu,Horror Thriller Fantasy,6.9,41
341,0.107147,Percy Jackson & the Olympians: The Lightning T...,Adventure Fantasy Family,6.0,2010
1939,0.105142,BloodRayne,Action Adventure Fantasy Horror,3.5,118
501,0.102377,The Little Prince,Adventure Animation Fantasy,7.6,756


In [ ]:
#### Hybrid similarity (text + votes) instead of only TF-IDF

In [ ]:
# Numeric similarity from vote features (for hybrid fusion)

import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity


vote_avg = df["vote_average"].fillna(df["vote_average"].median())   # fill missing with median
vote_cnt_log = np.log1p(df["vote_count"].fillna(0).clip(lower=0))   # use logip to compress scale so big blockbuster dont dominate EX: log1p(0)=0, log1p(10)=~2.4, log1p(10,000)=~9.2


num_feat = np.vstack([vote_avg.values, vote_cnt_log.values]).T  # Build a matrix with two columns

# NEED TO SCALE SO LARGER RANGE FEATURE DONT DOMINATE SIMILARITIES ( vote_average<<log1p(vote_count) on the same scale)

scaler = MinMaxScaler()
num_feat_scaled = scaler.fit_transform(num_feat)

# Compute numeric similarity (cosine in 2D)
# This gives an (n_movies x n_movies) matrix we can look how close they are
num_sim = cosine_similarity(num_feat_scaled)

print("Numeric features shape:", num_feat_scaled.shape) #4803 movies and 2 numeric features(vote avg, logip(vote count))
print("num_sim shape:", num_sim.shape, "| example range:", float(num_sim.min()), "to", float(num_sim.max()))


Numeric features shape: (4803, 2)
num_sim shape: (4803, 4803) | example range: 0.0 to 1.0000000000000004


In [ ]:
# Hybrid recommender:
# final_score = alpha * text_similarity + beta * numeric_similarity

# text_similarity comes from TF-IDF + cosine (via nn_model kneighbors)
# numeric_similarity comes from vote_average + log1p(vote_count)

import numpy as np

def hybrid_recommend_by_title(title, k=10, alpha=0.8, beta=0.2, candidate_pool=200):
    """
    Recommend top-k movies by blending text and numeric similarity.
    - alpha: weight for TF-IDF cosine (text)
    - beta : weight for numeric sim (votes)
    - candidate_pool: we first fetch top-N by text, then re-rank with hybrid score
    """
    assert abs(alpha + beta - 1.0) < 1e-6, "alpha + beta must sum to 1.0"

    idx = title_to_index.get(str(title).lower())
    if idx is None:
        raise ValueError(f"Title '{title}' not found.")

    # Get top-N by TEXT similarity (cosine) as the candidate set
    dists, inds = nn_model.kneighbors(X_tfidf[idx], n_neighbors=min(candidate_pool+1, X_tfidf.shape[0]))
    inds, dists = inds[0], dists[0]

    # Remove the item itself
    mask = inds != idx
    inds, dists = inds[mask], dists[mask]

    # Text similarity = 1 - cosine distance (already in ~[0,1])
    text_sim = 1.0 - dists

    # Numeric similarity from precomputed matrix (Step 7.1)
    num_sim_vec = num_sim[idx, inds]

    # Hybrid fusion
    final = alpha * text_sim + beta * num_sim_vec

    # Rank by final score
    order = np.argsort(-final)[:k]
    top_inds = inds[order]
    top_final = final[order]
    top_text = text_sim[order]
    top_num  = num_sim_vec[order]

    out = df.iloc[top_inds][["title","genres","vote_average","vote_count"]].copy()
    out.insert(0, "hybrid_score", top_final)
    out.insert(1, "text_sim",     top_text)
    out.insert(2, "num_sim",      top_num)
    return out


In [ ]:
# Example 1
print("Hybrid Recommendations for 'Avatar':")
display(hybrid_recommend_by_title("Avatar", k=10, alpha=0.8, beta=0.2))

# Example 2
print("Hybrid Recommendations for 'The Dark Knight':")
display(hybrid_recommend_by_title("The Dark Knight", k=10, alpha=0.8, beta=0.2))

# Example 3 (search + recommend)
search_term = "Harry Potter"
matches = [t for t in df["title"] if search_term.lower() in str(t).lower()]
print("Matches found for:", search_term, "->", matches[:10])
if matches:
    display(hybrid_recommend_by_title(matches[0], k=10, alpha=0.8, beta=0.2))


Hybrid Recommendations for 'Avatar':


,hybrid_score,text_sim,num_sim,title,genres,vote_average,vote_count
1341,0.345089,0.198955,0.929622,The Inhabited Island,Action Fantasy Science Fiction Thriller,5.3,23
634,0.340182,0.175673,0.998219,The Matrix,Action Science Fiction,7.9,8907
3604,0.323180,0.154280,0.998778,Apollo 18,Horror Thriller Science Fiction,5.0,356
529,0.313020,0.143492,0.991132,Tears of the Sun,Action Drama War,6.4,573
775,0.305279,0.134419,0.988720,Supernova,Horror Science Fiction Thriller,4.9,109
2130,0.303665,0.130784,0.995192,The American,Crime Drama Thriller,5.8,481
107,0.285958,0.107551,0.999586,Warcraft,Action Adventure Fantasy,6.3,2268
311,0.285273,0.107200,0.997567,The Adventures of Pluto Nash,Action Comedy Science Fiction,4.4,142
36,0.282095,0.102727,0.999569,Transformers: Age of Extinction,Science Fiction Action Adventure,5.8,3095
1274,0.280923,0.102275,0.995515,Just Visiting,Comedy Fantasy Science Fiction,4.8,171


Hybrid Recommendations for 'The Dark Knight':


,hybrid_score,text_sim,num_sim,title,genres,vote_average,vote_count
3,0.490777,0.363535,0.999745,The Dark Knight Rises,Action Crime Drama Thriller,7.6,9106
299,0.417602,0.273210,0.995169,Batman Forever,Action Crime Fantasy,5.2,1498
428,0.382599,0.228259,0.999959,Batman Returns,Action Fantasy,6.6,1673
1359,0.356420,0.195593,0.999728,Batman,Fantasy Action,7.0,2096
119,0.356254,0.195356,0.999847,Batman Begins,Action Crime Drama,7.5,7359
3854,0.338195,0.177763,0.979922,"Batman: The Dark Knight Returns, Part 2",Action Animation,7.9,419
879,0.320105,0.150584,0.998188,Law Abiding Citizen,Drama Crime Thriller,7.2,1486
205,0.314669,0.143364,0.999889,Sherlock Holmes: A Game of Shadows,Adventure Action Crime Mystery,7.0,3886
1181,0.309875,0.140564,0.987117,JFK,Drama Thriller History,7.5,502
210,0.304063,0.134574,0.982017,Batman & Robin,Action Crime Fantasy,4.2,1418


Matches found for: Harry Potter -> ['Harry Potter and the Half-Blood Prince', 'Harry Potter and the Order of the Phoenix', 'Harry Potter and the Goblet of Fire', 'Harry Potter and the Prisoner of Azkaban', "Harry Potter and the Philosopher's Stone", 'Harry Potter and the Chamber of Secrets']


,hybrid_score,text_sim,num_sim,title,genres,vote_average,vote_count
114,0.541176,0.426471,0.999995,Harry Potter and the Goblet of Fire,Adventure Fantasy Family,7.5,5608
113,0.444159,0.305199,0.999998,Harry Potter and the Order of the Phoenix,Adventure Fantasy Family Mystery,7.4,5494
191,0.368787,0.211007,0.999908,Harry Potter and the Prisoner of Azkaban,Adventure Fantasy Family,7.7,5877
276,0.367480,0.209354,0.999986,Harry Potter and the Chamber of Secrets,Adventure Fantasy Family,7.4,5815
197,0.347142,0.183938,0.999958,Harry Potter and the Philosopher's Stone,Adventure Fantasy Family,7.5,7006
4133,0.292770,0.124199,0.967051,Plush,Thriller,5.4,40
341,0.285527,0.107147,0.999046,Percy Jackson & the Olympians: The Lightning T...,Adventure Fantasy Family,6.0,2010
1939,0.283491,0.105142,0.996884,BloodRayne,Action Adventure Fantasy Horror,3.5,118
2979,0.281521,0.102003,0.999596,Bring It On,Comedy,5.8,568
501,0.279908,0.102377,0.990035,The Little Prince,Adventure Animation Fantasy,7.6,756


In [ ]:
####   train SVM + print metrics. Next step after that will be using SVM to gate recommendations.

In [ ]:
# SVM (multi-label) genre classifier

# Build multi-label targets from genres_display (protect multi-word names)
# Train a One-vs-Rest Linear SVM on the SAME kind of TF-IDF features (fit on TRAIN only to avoid leakage)
# Print micro/macro precision/recall/F1 + accuracy

import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer

# build genre labels from display column, preserving multi-word names
MULTIWORD = [("Science Fiction","Science_Fiction"), ("TV Movie","TV_Movie"), ("Film Noir","Film_Noir")]

def protect_phrases(s: str):
    s = str(s)
    for a,b in MULTIWORD:
        s = re.sub(rf"\b{re.escape(a)}\b", b, s, flags=re.IGNORECASE)
    return s

def split_genre_labels(s: str):
    # Protect multi-word genres by underscoring, then split by whitespace
    # then map underscores back to spaces for the final label names
    s = protect_phrases(s)
    tokens = [t for t in s.split() if t.strip()]
    labels = [t.replace("_"," ") for t in tokens]
    return labels

df["genre_list_svm"] = df["genres_display"].fillna("").apply(split_genre_labels)

# Keep only rows that have at least one genre label
mask_labeled = df["genre_list_svm"].apply(len) > 0
X_df = df.loc[mask_labeled].copy()
Y_list = X_df["genre_list_svm"].tolist()

# FEATURES: build a fresh TF-IDF (train-only fit) with the same columns you used before
# IMPORTANT: use 'genres_tfidf' (underscored for multi-word) for the genre text feature, not 'genres'/'genres_display'

weights_svm = {
    "overview":     0.6,
    "genres_tfidf": 0.2,
    "keywords":     0.15,
    "cast":         0.03,
    "director":     0.02
}
transformers_svm = []
for col in weights_svm.keys():
    transformers_svm.append((
        f"tfidf_{col}",
        TfidfVectorizer(stop_words="english", ngram_range=(1,2), min_df=2, max_features=50000),
        col
    ))
vectorizer_svm = ColumnTransformer(
    transformers_svm,
    transformer_weights={f"tfidf_{c}": w for c,w in weights_svm.items()}
)

# Train/test split on the labeled subset
X_train_df, X_test_df, y_train_list, y_test_list = train_test_split(
    X_df, Y_list, test_size=0.2, random_state=42, shuffle=True
)

# Fit TF-IDF on TRAIN ONLY to avoid leakage, then transform train & test
X_train_vec = vectorizer_svm.fit_transform(X_train_df)
X_test_vec  = vectorizer_svm.transform(X_test_df)

# TRAIN SVM: One-vs-Rest LinearSVC with class balancing
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(y_train_list)
y_test  = mlb.transform(y_test_list)

svm_clf = OneVsRestClassifier(LinearSVC(class_weight="balanced", random_state=42))
svm_clf.fit(X_train_vec, y_train)
y_pred = svm_clf.predict(X_test_vec)

# METRICS:
acc   = accuracy_score(y_test, y_pred)
micro_p = precision_score(y_test, y_pred, average="micro", zero_division=0)
micro_r = recall_score(y_test, y_pred, average="micro", zero_division=0)
micro_f = f1_score(y_test, y_pred, average="micro", zero_division=0)
macro_p = precision_score(y_test, y_pred, average="macro", zero_division=0)
macro_r = recall_score(y_test, y_pred, average="macro", zero_division=0)
macro_f = f1_score(y_test, y_pred, average="macro", zero_division=0)

print("SVM (multi-label) metrics")
print(f"  Accuracy      : {acc:.3f}")
print(f"  Micro Precision/Recall/F1: {micro_p:.3f} / {micro_r:.3f} / {micro_f:.3f}")
print(f"  Macro Precision/Recall/F1: {macro_p:.3f} / {macro_r:.3f} / {macro_f:.3f}")

# Optional: brief per-genre table (top 12 by support)
report = classification_report(y_test, y_pred, target_names=mlb.classes_, output_dict=True, zero_division=0)
rep_df = (pd.DataFrame(report).transpose()
          .reset_index()
          .rename(columns={"index":"genre"})
          .sort_values(by="support", ascending=False))
display(rep_df.head(12)[["genre","precision","recall","f1-score","support"]])


SVM (multi-label) metrics
  Accuracy      : 0.975
  Micro Precision/Recall/F1: 1.000 / 0.990 / 0.995
  Macro Precision/Recall/F1: 0.952 / 0.919 / 0.932


,genre,precision,recall,f1-score,support
21,micro avg,0.999592,0.990307,0.994928,2476.0
24,samples avg,0.999480,0.993965,0.996175,2476.0
23,weighted avg,0.999190,0.990307,0.994473,2476.0
22,macro avg,0.952278,0.918778,0.932377,2476.0
6,Drama,0.997835,0.991398,0.994606,465.0
3,Comedy,1.000000,0.996894,0.998445,322.0
17,Thriller,1.000000,0.992593,0.996283,270.0
0,Action,1.000000,0.996016,0.998004,251.0
14,Romance,1.000000,0.994012,0.996997,167.0
4,Crime,1.000000,0.981366,0.990596,161.0


In [ ]:
# SVM-gated hybrid recommender

def svm_predict_genres_for_title(title: str):
    """Predict genres for a given title using the trained SVM classifier."""
    idx = title_to_index.get(title.lower())
    if idx is None:
        raise ValueError(f"Title '{title}' not found.")
    x_vec = vectorizer_svm.transform(df.iloc[[idx]])
    pred_bin = svm_clf.predict(x_vec)
    return [mlb.classes_[i] for i,v in enumerate(pred_bin[0]) if v == 1]

def gated_hybrid_recommend_by_title(title, k=10, alpha=0.8, beta=0.2, candidate_pool=200):
    idx = title_to_index.get(str(title).lower())
    if idx is None:
        raise ValueError(f"Title '{title}' not found.")

    # get SVM-predicted genres for the query movie
    pred_genres = svm_predict_genres_for_title(title)

    # get top-N by text sim (candidate pool)
    dists, inds = nn_model.kneighbors(X_tfidf[idx], n_neighbors=min(candidate_pool+1, X_tfidf.shape[0]))
    inds, dists = inds[0], dists[0]
    mask = inds != idx
    inds, dists = inds[mask], dists[mask]
    text_sim = 1.0 - dists

    # filter candidates by genre overlap with SVM prediction
    filtered_inds = []
    filtered_text = []
    for j, sim in zip(inds, text_sim):
        movie_genres = df.iloc[j]["genre_list_svm"]
        if any(g in pred_genres for g in movie_genres):
            filtered_inds.append(j)
            filtered_text.append(sim)

    if not filtered_inds:
        print("Warning: no candidates survived SVM gating; falling back to ungated hybrid.")
        return hybrid_recommend_by_title(title, k, alpha, beta, candidate_pool)

    filtered_inds = np.array(filtered_inds)
    filtered_text = np.array(filtered_text)

    # numeric similarity
    num_sim_vec = num_sim[idx, filtered_inds]

    # hybrid fusion
    final = alpha * filtered_text + beta * num_sim_vec
    order = np.argsort(-final)[:k]
    top_inds = filtered_inds[order]
    top_final = final[order]

    out = df.iloc[top_inds][["title","genres","vote_average","vote_count"]].copy()
    out.insert(0, "hybrid_score", top_final)
    return out

# Example runs
print("Gated Hybrid Recommendations for 'Avatar':")
display(gated_hybrid_recommend_by_title("Avatar", k=10))

print("Gated Hybrid Recommendations for 'The Dark Knight':")
display(gated_hybrid_recommend_by_title("The Dark Knight", k=10))


Gated Hybrid Recommendations for 'Avatar':


,hybrid_score,title,genres,vote_average,vote_count
1341,0.345089,The Inhabited Island,Action Fantasy Science Fiction Thriller,5.3,23
634,0.340182,The Matrix,Action Science Fiction,7.9,8907
3604,0.323180,Apollo 18,Horror Thriller Science Fiction,5.0,356
529,0.313020,Tears of the Sun,Action Drama War,6.4,573
775,0.305279,Supernova,Horror Science Fiction Thriller,4.9,109
107,0.285958,Warcraft,Action Adventure Fantasy,6.3,2268
311,0.285273,The Adventures of Pluto Nash,Action Comedy Science Fiction,4.4,142
36,0.282095,Transformers: Age of Extinction,Science Fiction Action Adventure,5.8,3095
1274,0.280923,Just Visiting,Comedy Fantasy Science Fiction,4.8,171
420,0.280662,Hellboy II: The Golden Army,Adventure Fantasy Science Fiction,6.5,1527


Gated Hybrid Recommendations for 'The Dark Knight':


,hybrid_score,title,genres,vote_average,vote_count
3,0.490777,The Dark Knight Rises,Action Crime Drama Thriller,7.6,9106
299,0.417602,Batman Forever,Action Crime Fantasy,5.2,1498
428,0.382599,Batman Returns,Action Fantasy,6.6,1673
1359,0.356420,Batman,Fantasy Action,7.0,2096
119,0.356254,Batman Begins,Action Crime Drama,7.5,7359
3854,0.338195,"Batman: The Dark Knight Returns, Part 2",Action Animation,7.9,419
879,0.320105,Law Abiding Citizen,Drama Crime Thriller,7.2,1486
205,0.314669,Sherlock Holmes: A Game of Shadows,Adventure Action Crime Mystery,7.0,3886
1181,0.309875,JFK,Drama Thriller History,7.5,502
210,0.304063,Batman & Robin,Action Crime Fantasy,4.2,1418


In [ ]:
##   EVALUATION

In [ ]:
# Eval: Precision@k (genre-overlap) + NDCG@k for 4 models:
# 1.Cosine (nn_model)   2. KNN (knn_model)
# 3.Hybrid (alpha,beta) 4. SVM-gated Hybrid

import numpy as np


K = 10
ALPHA, BETA = 0.8, 0.2         # same weights you used in hybrid
CANDIDATE_POOL = 200           # same pool you used in hybrid
assert abs(ALPHA + BETA - 1.0) < 1e-6

# helpers
def ndcg_at_k(rels, k):
    rels = np.array(rels[:k], dtype=float)
    dcg = np.sum(rels / np.log2(np.arange(2, len(rels)+2)))
    R = int(rels.sum())
    idcg = np.sum(np.ones(min(R, k)) / np.log2(np.arange(2, min(R, k)+2))) if R > 0 else 0.0
    return (dcg / idcg) if idcg > 0 else 0.0

def precision_at_k(rels, k):
    rels = np.array(rels[:k], dtype=float)
    return float(rels.mean()) if len(rels) > 0 else 0.0

def rel_vector(q_idx, indices):
    """Binary relevance for neighbors: 1 if any genre overlaps with query genres."""
    qg = set(df.iloc[q_idx]["genre_list_svm"])
    rels = []
    for j in indices:
        gj = set(df.iloc[j]["genre_list_svm"])
        rels.append(1 if (qg & gj) else 0)
    return rels

def topk_cosine_indices(i, k):
    d, ind = nn_model.kneighbors(X_tfidf[i], n_neighbors=min(k+1, X_tfidf.shape[0]))
    ind = ind[0]
    ind = ind[ind != i][:k]
    return ind

def topk_knn_indices(i, k):
    d, ind = knn_model.kneighbors(X_tfidf[i], n_neighbors=min(k+1, X_tfidf.shape[0]))
    ind = ind[0]
    ind = ind[ind != i][:k]
    return ind

def topk_hybrid_indices(i, k, alpha=ALPHA, beta=BETA, pool=CANDIDATE_POOL):
    d, ind = nn_model.kneighbors(X_tfidf[i], n_neighbors=min(pool+1, X_tfidf.shape[0]))
    ind, d = ind[0], d[0]
    mask = ind != i
    ind, d = ind[mask], d[mask]
    text_sim = 1.0 - d
    num_vec  = num_sim[i, ind]
    final = alpha * text_sim + beta * num_vec
    order = np.argsort(-final)[:k]
    return ind[order]

def topk_gated_hybrid_indices(i, k, alpha=ALPHA, beta=BETA, pool=CANDIDATE_POOL):
    # predict genres for query i
    x_vec = vectorizer_svm.transform(df.iloc[[i]])
    pred = svm_clf.predict(x_vec)[0]
    pred_genres = {mlb.classes_[t] for t,v in enumerate(pred) if v == 1}

    d, ind = nn_model.kneighbors(X_tfidf[i], n_neighbors=min(pool+1, X_tfidf.shape[0]))
    ind, d = ind[0], d[0]
    mask = ind != i
    ind, d = ind[mask], d[mask]
    text_sim = 1.0 - d

    # filter by SVM-predicted genres
    keep = []
    keep_text = []
    for j, s in zip(ind, text_sim):
        gj = set(df.iloc[j]["genre_list_svm"])
        if gj & pred_genres:
            keep.append(j); keep_text.append(s)
    if not keep:                   # fallback if nothing passes the gate
        return topk_hybrid_indices(i, k, alpha, beta, pool)

    keep = np.array(keep); keep_text = np.array(keep_text)
    num_vec = num_sim[i, keep]
    final = alpha * keep_text + beta * num_vec
    order = np.argsort(-final)[:k]
    return keep[order]

# evaluate over items that have at least one genre
idx_eval = [i for i, gs in enumerate(df["genre_list_svm"]) if len(gs) > 0]

def eval_model(topk_fn, name):
    precs, ndcgs = [], []
    for i in idx_eval:
        neigh = topk_fn(i, K)
        rels = rel_vector(i, neigh)
        precs.append(precision_at_k(rels, K))
        ndcgs.append(ndcg_at_k(rels, K))
    print(f"{name} | items: {len(idx_eval)}  |  Precision@{K}: {np.mean(precs):.3f}  |  NDCG@{K}: {np.mean(ndcgs):.3f}")

eval_model(topk_cosine_indices,        "Cosine (TF-IDF)")
eval_model(topk_knn_indices,           "KNN (TF-IDF)")
eval_model(topk_hybrid_indices,        "Hybrid (text+votes)")
eval_model(topk_gated_hybrid_indices,  "Gated Hybrid (SVM + hybrid)")


Cosine (TF-IDF) | items: 4803  |  Precision@10: 0.897  |  NDCG@10: 0.954
KNN (TF-IDF) | items: 4803  |  Precision@10: 0.897  |  NDCG@10: 0.954
Hybrid (text+votes) | items: 4803  |  Precision@10: 0.903  |  NDCG@10: 0.956
Gated Hybrid (SVM + hybrid) | items: 4803  |  Precision@10: 1.000  |  NDCG@10: 1.000


In [ ]:
### KMeans #######

In [ ]:
# LSA (TruncatedSVD) on TF-IDF for clustering

import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

# Choose latent dimensionality for LSA. 100–300 sweet spot

n_components = 200

# Fit LSA on the *same* TF-IDF matrix already built (X_tfidf)

svd_lsa = TruncatedSVD(n_components=n_components, random_state=42)
X_lsa = svd_lsa.fit_transform(X_tfidf)   # shape: (n_movies, n_components)


expl_var = svd_lsa.explained_variance_ratio_
cum_var = np.cumsum(expl_var)

print(f"TF-IDF shape: {X_tfidf.shape}  ->  LSA shape: {X_lsa.shape}")
print(f"Cumulative explained variance with {n_components} comps: {cum_var[-1]:.3f}")

# K-Means uses Euclidean distance, so scaling matters

scaler_lsa = StandardScaler()
X_lsa_z = scaler_lsa.fit_transform(X_lsa)

print("Standardized LSA ready for K-Means: X_lsa_z shape =", X_lsa_z.shape)


TF-IDF shape: (4803, 32669)  ->  LSA shape: (4803, 200)
Cumulative explained variance with 200 comps: 0.203
Standardized LSA ready for K-Means: X_lsa_z shape = (4803, 200)


In [ ]:
# K-Means clustering on LSA-reduced TF-IDF

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

k_values = [5, 10, 15, 20, 25]

results = {}
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_lsa_z)

    sil = silhouette_score(X_lsa_z, labels)
    results[k] = sil
    print(f"k={k:2d} | silhouette={sil:.3f}")

# Choose the best k (highest silhouette)
best_k = max(results, key=results.get)
print(f"\nBest k by silhouette score: {best_k} (score={results[best_k]:.3f})")

# Refit KMeans with the chosen k and store labels
kmeans_final = KMeans(n_clusters=best_k, random_state=42, n_init=10)
cluster_labels = kmeans_final.fit_predict(X_lsa_z)

# Attach cluster labels to your dataframe
df["cluster"] = cluster_labels
df[["title", "genres", "cluster"]].head(10)


k= 5 | silhouette=-0.016
k=10 | silhouette=-0.011
k=15 | silhouette=-0.011
k=20 | silhouette=-0.004
k=25 | silhouette=-0.000

Best k by silhouette score: 25 (score=-0.000)


,title,genres,cluster
0,Avatar,Action Adventure Fantasy Science Fiction,19
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,19
2,Spectre,Action Adventure Crime,9
3,The Dark Knight Rises,Action Crime Drama Thriller,4
4,John Carter,Action Adventure Science Fiction,16
5,Spider-Man 3,Fantasy Action Adventure,19
6,Tangled,Animation Family,9
7,Avengers: Age of Ultron,Action Adventure Science Fiction,16
8,Harry Potter and the Half-Blood Prince,Adventure Fantasy Family,19
9,Batman v Superman: Dawn of Justice,Action Adventure Fantasy,19


In [ ]:
# Interpret clusters: top genres & top TF-IDF terms per cluster

import numpy as np
import pandas as pd
from collections import Counter

assert "cluster" in df.columns, "Run Step C2 first to add df['cluster']"
assert X_tfidf.shape[0] == df.shape[0], "X_tfidf and df must align (same number of rows)"

# Try to get feature names from your ColumnTransformer (vectorizer) used for X_tfidf
try:
    feature_names = vectorizer.get_feature_names_out()
except Exception as e:
    feature_names = None
    print("Warning: could not extract feature names from your ColumnTransformer. "
          "Top-term display will fall back to indices only.", e)

def _pretty_feature_name(name: str) -> str:
    """
    Clean up ColumnTransformer feature names for display.
    Names often look like 'tfidf_overview__space_ship' or 'tfidf_genres_tfidf__Science_Fiction'.
    We strip the 'tfidf_xxx__' prefix and turn underscores into spaces for readability.
    """
    s = str(name)
    if "__" in s:
        s = s.split("__", 1)[1]
    # Readability: Science_Fiction --> Science Fiction (it's OK for display)
    s = s.replace("_", " ")
    return s

def cluster_top_terms(X, labels, c, n_terms=15):
    """
    Compute top TF-IDF features for cluster c by averaging member rows in X (sparse).
    Returns list of (term, weight) pairs for the top n_terms features.
    """
    rows = np.where(labels == c)[0]
    if rows.size == 0:
        return []
    centroid = X[rows].mean(axis=0)                # 1 x D sparse row
    # Convert to 1D dense array safely
    if hasattr(centroid, "A1"):
        weights = centroid.A1
    else:
        weights = np.asarray(centroid).ravel()
    top_idx = np.argsort(-weights)[:n_terms]
    if feature_names is not None:
        terms = [_pretty_feature_name(feature_names[i]) for i in top_idx]
    else:
        terms = [f"feat_{i}" for i in top_idx]
    return list(zip(terms, weights[top_idx]))

def cluster_top_genres(df_cluster, topn=8):
    """
    Count genres inside a cluster using your parsed genre list.
    If you have df['genre_list_svm'] (list of labels), we prefer that.
    Otherwise we split df['genres'] by whitespace as a fallback.
    """
    if "genre_list_svm" in df_cluster.columns:
        all_genres = [g for row in df_cluster["genre_list_svm"] for g in row]
    else:
        # fallback: split the display string on whitespace
        all_genres = []
        for s in df_cluster["genres"].fillna(""):
            all_genres.extend([t for t in str(s).split() if t])
    counts = Counter(all_genres)
    return counts.most_common(topn)

def show_cluster_summary(c, n_terms=15, n_titles=8):
    """
    Print a human-readable summary for a single cluster:
      - size
      - top genres (counts)
      - top TF-IDF terms (weights)
      - a few sample movie titles
    """
    labels = df["cluster"].values
    rows = np.where(labels == c)[0]
    sub = df.iloc[rows]

    print("="*80)
    print(f"Cluster {c}  |  size={len(sub)}")
    print("- Top genres:")
    for g, cnt in cluster_top_genres(sub, topn=10):
        print(f"   {g:20s}  {cnt}")
    print("- Top TF-IDF terms:")
    for term, w in cluster_top_terms(X_tfidf, labels, c, n_terms=n_terms):
        print(f"   {term:30s} {w:.4f}")
    print("- Sample titles:")
    for t in sub["title"].head(n_titles).tolist():
        print(f"   • {t}")
    print("="*80)


cluster_sizes = df["cluster"].value_counts().sort_index()
print("Cluster sizes:\n", cluster_sizes.to_string())

# Show summaries for the first few clusters
n_clusters = df["cluster"].nunique()
for c in range(min(n_clusters, 5)):  # change 5 -> n_clusters if you want all
    show_cluster_summary(c, n_terms=15, n_titles=8)



Cluster sizes:
 cluster
0       95
1      117
2      153
3      255
4      309
5      109
6      338
7       77
8      293
9     1370
10      77
11      91
12     155
13      78
14      72
15     209
16     126
17     116
18      94
19      97
20      28
21     130
22      31
23     168
24     215
Cluster 0  |  size=95
- Top genres:
   Adventure             95
   Drama                 95
   Action                62
   Thriller              21
   Romance               21
   War                   14
   History               12
   Science Fiction       6
   Western               4
   Fantasy               3
- Top TF-IDF terms:
   adventure drama                0.1088
   adventure                      0.0644
   action adventure               0.0438
   drama                          0.0399
   action                         0.0344
   drama thriller                 0.0163
   drama action                   0.0133
   war                            0.0123
   romance                        0.0122

In [ ]:
# Cluster-aware recommendation function

def cluster_recommend_by_title(title, k=10):
    """
    Recommend movies restricted to the same cluster as the query movie.
    - title: input movie title
    - k: number of recommendations to return
    """
    # Get index of the query movie
    idx = title_to_index.get(str(title).lower())
    if idx is None:
        raise ValueError(f"Title '{title}' not found.")

    # Find which cluster this movie belongs to
    query_cluster = df.loc[idx, "cluster"]

    # Candidate pool = all movies in the same cluster
    candidates = df[df["cluster"] == query_cluster].index.tolist()

    # Compute cosine similarity within that cluster (excluding the movie itself)
    dists, inds = nn_model.kneighbors(X_tfidf[idx], n_neighbors=len(candidates))
    inds, dists = inds[0], dists[0]

    # Filter to candidates only & remove self
    results = []
    for dist, ind in zip(dists, inds):
        if ind != idx and ind in candidates:
            sim = 1.0 - dist   # cosine similarity
            results.append((ind, sim))

    # Sort by similarity and keep top-k
    results = sorted(results, key=lambda x: -x[1])[:k]

    # Build dataframe output
    out = df.iloc[[i for i, _ in results]][["title","genres","vote_average","vote_count"]].copy()
    out.insert(0, "similarity", [s for _, s in results])
    out.insert(1, "cluster", query_cluster)
    return out


print("Cluster-aware Recommendations for 'Avatar':")
display(cluster_recommend_by_title("Avatar", k=10))

print("Cluster-aware Recommendations for 'The Dark Knight':")
display(cluster_recommend_by_title("The Dark Knight", k=10))


Cluster-aware Recommendations for 'Avatar':


,similarity,cluster,title,genres,vote_average,vote_count
107,0.107551,19,Warcraft,Action Adventure Fantasy,6.3,2268
420,0.101436,19,Hellboy II: The Golden Army,Adventure Fantasy Science Fiction,6.5,1527
3494,0.094429,19,Beastmaster 2: Through the Portal of Time,Action Adventure Fantasy Science Fiction,4.6,17
813,0.094384,19,Superman,Action Adventure Fantasy Science Fiction,6.9,1022
870,0.094384,19,Superman II,Action Adventure Fantasy Science Fiction,6.5,629
14,0.094384,19,Man of Steel,Action Adventure Fantasy Science Fiction,6.5,6359
46,0.094384,19,X-Men: Days of Future Past,Action Adventure Fantasy Science Fiction,7.5,6032
369,0.094067,19,Lara Croft Tomb Raider: The Cradle of Life,Action Adventure Fantasy Thriller,5.5,1418
199,0.088792,19,Pirates of the Caribbean: The Curse of the Bla...,Adventure Fantasy Action,7.5,6985
419,0.084963,19,Jumper,Adventure Fantasy Science Fiction,5.9,1799


Cluster-aware Recommendations for 'The Dark Knight':


,similarity,cluster,title,genres,vote_average,vote_count
3854,0.177763,9,"Batman: The Dark Knight Returns, Part 2",Action Animation,7.9,419
205,0.143364,9,Sherlock Holmes: A Game of Shadows,Adventure Action Crime Mystery,7.0,3886
2371,0.119800,9,RockNRolla,Action Crime Thriller,6.9,836
4135,0.114157,9,Gangster's Paradise: Jerusalema,Drama Action Crime Foreign,6.8,16
58,0.109820,9,Rush Hour 3,Action Comedy Crime Thriller,6.1,783
1479,0.104444,9,Exit Wounds,Action Crime Thriller,5.3,110
3819,0.101462,9,Defendor,Drama Action Comedy Crime,6.5,195
1133,0.098068,9,15 Minutes,Action Crime Thriller,5.7,191
992,0.097894,9,Domino,Action Crime,6.0,444
963,0.096908,9,Ninja Assassin,Action Crime Thriller,6.2,371


In [ ]:
# Unified Evaluation Block: 5 Models (Retrieval, Classifier, Clustering)

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import silhouette_score


K = 10
ALPHA, BETA = 0.8, 0.2
assert abs(ALPHA + BETA - 1.0) < 1e-6

def precision_at_k(rels, k):
    rels = np.array(rels[:k], dtype=float)
    return float(rels.mean()) if len(rels) else 0.0

def ndcg_at_k(rels, k):
    rels = np.array(rels[:k], dtype=float)
    dcg = np.sum(rels / np.log2(np.arange(2, len(rels)+2)))
    R = int(rels.sum())
    idcg = np.sum(np.ones(min(R, k)) / np.log2(np.arange(2, min(R, k)+2))) if R > 0 else 0.0
    return (dcg / idcg) if idcg > 0 else 0.0

def rel_vector(q_idx, indices):
    """Binary relevance via genre overlap with query movie."""
    qg = set(df.iloc[q_idx]["genre_list_svm"])
    rels = []
    for j in indices:
        gj = set(df.iloc[j]["genre_list_svm"])
        rels.append(1 if (qg & gj) else 0)
    return rels

idx_eval = [i for i, gs in enumerate(df["genre_list_svm"]) if len(gs) > 0]


# Top-k indexers for retrieval models

def topk_cosine_indices(i, k):
    d, ind = nn_model.kneighbors(X_tfidf[i], n_neighbors=min(k+1, X_tfidf.shape[0]))
    ind = ind[0]
    ind = ind[ind != i][:k]
    return ind

def topk_knn_indices(i, k):
    d, ind = knn_model.kneighbors(X_tfidf[i], n_neighbors=min(k+1, X_tfidf.shape[0]))
    ind = ind[0]
    ind = ind[ind != i][:k]
    return ind

def topk_hybrid_indices(i, k, alpha=ALPHA, beta=BETA, pool=200):
    d, ind = nn_model.kneighbors(X_tfidf[i], n_neighbors=min(pool+1, X_tfidf.shape[0]))
    ind, d = ind[0], d[0]
    mask = ind != i
    ind, d = ind[mask], d[mask]
    text_sim = 1.0 - d
    num_vec  = num_sim[i, ind]
    final = alpha * text_sim + beta * num_vec
    order = np.argsort(-final)[:k]
    return ind[order]

def topk_cluster_only_indices(i, k, min_vote_count=50):
    """
    Pure clustering recommender (no TF-IDF ranking):
    - candidates = same cluster as query (except itself)
    - filter by vote_count for quality
    - sort by (vote_average, vote_count) descending
    """
    my_cluster = df.loc[i, "cluster"]
    cand = df.index[(df["cluster"] == my_cluster) & (df.index != i)].tolist()
    # quality filter
    if "vote_count" in df.columns:
        cand = [j for j in cand if pd.notna(df.loc[j, "vote_count"]) and df.loc[j, "vote_count"] >= min_vote_count]
    if not cand:
        return np.array([], dtype=int)
    # sort by vote_average then vote_count (desc)
    cand_sorted = sorted(
        cand,
        key=lambda j: (float(df.loc[j, "vote_average"]) if pd.notna(df.loc[j, "vote_average"]) else 0.0,
                       float(df.loc[j, "vote_count"]) if pd.notna(df.loc[j, "vote_count"]) else 0.0),
        reverse=True
    )
    return np.array(cand_sorted[:k], dtype=int)

def eval_model(topk_fn, name):
    precs, ndcgs = [], []
    for i in idx_eval:
        neigh = topk_fn(i, K)
        if len(neigh) == 0:
            continue
        rels = rel_vector(i, neigh)
        precs.append(precision_at_k(rels, K))
        ndcgs.append(ndcg_at_k(rels, K))
    print(f"{name:<28s} | items: {len(precs):4d} | Precision@{K}: {np.mean(precs):.3f} | NDCG@{K}: {np.mean(ndcgs):.3f}")


# (1–4) Retrieval models

print(" Retrieval Models (Precision@K and NDCG@K)")
eval_model(topk_cosine_indices,       "TF-IDF + Cosine")
eval_model(topk_knn_indices,          "KNN (TF-IDF space)")
eval_model(topk_hybrid_indices,       "Hybrid (text + votes)")
eval_model(topk_cluster_only_indices, "Cluster-only (same cluster)")


# Classifier: SVM genres

print("\n Classifier (SVM)-multi-label genre prediction ")

try:
    _ = X_test_vec, y_test, y_pred  # already computed
except NameError:
    from sklearn.model_selection import train_test_split
    # build (or reuse) labels
    if "genre_list_svm" not in df.columns:
        raise RuntimeError("genre_list_svm not found; build it before SVM eval.")
    mask_labeled = df["genre_list_svm"].apply(len) > 0
    X_df = df.loc[mask_labeled].copy()
    Y_list = X_df["genre_list_svm"].tolist()
    # split & vectorize
    X_train_df, X_test_df, y_train_list, y_test_list = train_test_split(
        X_df, Y_list, test_size=0.2, random_state=42, shuffle=True
    )
    X_train_vec = vectorizer_svm.fit_transform(X_train_df)
    X_test_vec  = vectorizer_svm.transform(X_test_df)
    from sklearn.preprocessing import MultiLabelBinarizer
    mlb = MultiLabelBinarizer()
    y_train = mlb.fit_transform(y_train_list)
    y_test  = mlb.transform(y_test_list)
    from sklearn.multiclass import OneVsRestClassifier
    from sklearn.svm import LinearSVC
    svm_clf = OneVsRestClassifier(LinearSVC(class_weight="balanced", random_state=42))
    svm_clf.fit(X_train_vec, y_train)
    y_pred = svm_clf.predict(X_test_vec)

# Metrics
acc   = accuracy_score(y_test, y_pred)
micro_p = precision_score(y_test, y_pred, average="micro", zero_division=0)
micro_r = recall_score(y_test, y_pred, average="micro", zero_division=0)
micro_f = f1_score(y_test, y_pred, average="micro", zero_division=0)
macro_p = precision_score(y_test, y_pred, average="macro", zero_division=0)
macro_r = recall_score(y_test, y_pred, average="macro", zero_division=0)
macro_f = f1_score(y_test, y_pred, average="macro", zero_division=0)
print(f"Accuracy: {acc:.3f} | Micro P/R/F1: {micro_p:.3f}/{micro_r:.3f}/{micro_f:.3f} | Macro P/R/F1: {macro_p:.3f}/{macro_r:.3f}/{macro_f:.3f}")



 Retrieval Models (genre-overlap@K and NDCG@K)
TF-IDF + Cosine              | items: 4803 | Precision@10: 0.897 | NDCG@10: 0.954
KNN (TF-IDF space)           | items: 4803 | Precision@10: 0.897 | NDCG@10: 0.954
Hybrid (text + votes)        | items: 4803 | Precision@10: 0.903 | NDCG@10: 0.956
Cluster-only (same cluster)  | items: 4775 | Precision@10: 0.815 | NDCG@10: 0.900

 Classifier (SVM)-multi-label genre prediction 
Accuracy: 0.975 | Micro P/R/F1: 1.000/0.990/0.995 | Macro P/R/F1: 0.952/0.919/0.932


In [ ]:
# Pretty summary table for retrieval models
import numpy as np
import pandas as pd

K = 10

def eval_model_table(topk_fn, name):
    precs, ndcgs = [], []
    counted = 0
    for i, gs in enumerate(df["genre_list_svm"]):
        if len(gs) == 0:
            continue
        neigh = topk_fn(i, K)
        if len(neigh) == 0:
            continue
        rels = [1 if set(df.iloc[i]["genre_list_svm"]) & set(df.iloc[j]["genre_list_svm"]) else 0 for j in neigh]
        # Precision@K
        p = float(np.mean(rels[:K]))
        # NDCG@K
        rels = np.array(rels[:K], dtype=float)
        dcg = np.sum(rels / np.log2(np.arange(2, len(rels)+2)))
        R = int(rels.sum())
        idcg = np.sum(np.ones(min(R, K)) / np.log2(np.arange(2, min(R, K)+2))) if R > 0 else 0.0
        ndcg = (dcg / idcg) if idcg > 0 else 0.0
        precs.append(p); ndcgs.append(ndcg); counted += 1
    return {"Model": name, f"Precision@{K}": np.mean(precs), f"NDCG@{K}": np.mean(ndcgs), "Items": counted}

rows = []
rows.append(eval_model_table(topk_cosine_indices,       "TF-IDF + Cosine"))
rows.append(eval_model_table(topk_knn_indices,          "KNN (TF-IDF space)"))
rows.append(eval_model_table(topk_hybrid_indices,       "Hybrid (text + votes)"))
rows.append(eval_model_table(topk_cluster_only_indices, "Cluster-only (same cluster)"))

metrics_df = pd.DataFrame(rows)
metrics_df = metrics_df.sort_values(by=[f"NDCG@{K}", f"Precision@{K}"], ascending=False).reset_index(drop=True)
display(metrics_df.style.format({f"Precision@{K}":"{:.3f}", f"NDCG@{K}":"{:.3f}", "Items":"{:,.0f}"}))


,Model,Precision@10,NDCG@10,Items
0,Hybrid (text + votes),0.903,0.956,"4,803"
1,TF-IDF + Cosine,0.897,0.954,"4,803"
2,KNN (TF-IDF space),0.897,0.954,"4,803"
3,Cluster-only (same cluster),0.815,0.900,"4,775"
